In [1]:
import tensorflow as tf
import numpy as np

In [2]:
train_label= np.loadtxt('train_label2.txt',delimiter=',', dtype=np.float32)
train_label=train_label.reshape(-1,1)

test_label= np.loadtxt('test_label2.txt',delimiter=',', dtype=np.float32)
test_label=test_label.reshape(-1,1)

train_img= np.loadtxt('train_img3.txt',delimiter=',', dtype=np.float32)

test_img= np.loadtxt('test_img3.txt',delimiter=',', dtype=np.float32)

train_img=np.transpose(train_img)
train_img=np.reshape(train_img,[-1,3*28*28])
test_img=np.transpose(test_img)
test_img=np.reshape(test_img,[-1,3*28*28])

In [3]:
train_img.shape

(10000, 2352)

In [15]:
batch_xs=np.zeros([50,3*28*28])
batch_ys=np.zeros([50,1])
for j in range(50):
    batch_xs[j,:] = train_img[np.random.choice(2000,1),:]
    batch_ys[j,:] = train_label[np.random.choice(2000,1),:]
    
print(batch_xs.shape)
print(batch_ys.shape)

(50, 2352)
(50, 1)


In [4]:
tf.reset_default_graph()
learning_rate = 0.001
training_epochs = 15
batch_size = 50

X=tf.placeholder(tf.float32, [None, 3*28*28])
X_img=tf.reshape(X, [-1,3, 28, 28])
X_img=tf.transpose(X_img, [0, 2, 3, 1])

Y=tf.placeholder(tf.int32, [None, 1])
Y_one_hot=tf.one_hot(Y, 2)
Y_one_hot=tf.reshape(Y_one_hot, [-1,2])
keep_prob = tf.placeholder(tf.float32)


W1 = tf.get_variable("W1", shape=[3,3,3,64], initializer=tf.contrib.layers.xavier_initializer())
L1=tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
L1=tf.nn.relu(L1)
L1=tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# L1 = [batch_size,14,14,32]

W2 = tf.get_variable("W2", shape=[3,3,64,128], initializer=tf.contrib.layers.xavier_initializer())
L2=tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[3,3,128,256], initializer=tf.contrib.layers.xavier_initializer())
L3=tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME')
L3=tf.nn.relu(L3)
L3=tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 256 * 4 * 4])

W4 = tf.get_variable("W4", shape=[256 * 4 * 4, 625],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[625, 2],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L4, W5) + b5


cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y_one_hot))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

print('Learning started.')

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(10000 / 100)

    for i in range(total_batch):
        #batch_xs, batch_ys = tf.train.batch([train_img, train_label], batch_size=50, enqueue_many=True)
        batch_xs=np.zeros([100,3*28*28])
        batch_ys=np.zeros([100,1])
        for j in range(100):
            batch_xs[j,:] = train_img[np.random.choice(10000,1),:]
            batch_ys[j,:] = train_label[np.random.choice(10000,1),:]
        
        #for j in np.random.choice(2000,50):
            #batch_xs = np.stack(train_img[j,:])
            #batch_ys = np.stack(train_label[i,:])
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        #c = cost.eval(session=sess, feed_dict={X: batch_xs, Y: batch_ys})
        #_ = optimizer.eval(session=sess, feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: test_img, Y: test_label, keep_prob: 1}))

Learning started.
Epoch: 0001 cost = 3.092314430
Epoch: 0002 cost = 0.332067181
Epoch: 0003 cost = 0.331962325
Epoch: 0004 cost = 0.321106450
Epoch: 0005 cost = 0.336569742
Epoch: 0006 cost = 0.337223149
Epoch: 0007 cost = 0.318352387
Epoch: 0008 cost = 0.333068678
Epoch: 0009 cost = 0.313460483
Epoch: 0010 cost = 0.318768267
Epoch: 0011 cost = 0.325595538
Epoch: 0012 cost = 0.321933556
Epoch: 0013 cost = 0.334010931
Epoch: 0014 cost = 0.327259651
Epoch: 0015 cost = 0.335359533
Learning Finished!
Accuracy: 0.925667
